In [1]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense, GlobalAveragePooling2D
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from imutils import paths
from keras.utils import to_categorical
import random
import cv2
import tensorflowjs as tfjs

AttributeError: module 'plaidml.op' has no attribute 'PoolDataFormat'

In [2]:
# initialize the number of epochs to train for, initial learning rate,
# and batch size
EPOCHS = 20
INIT_LR = 1e-3
BS = 32
IMG_DIMS = (224, 224, 3)

MODEL_NAME = "attractive"

data_set_dir = 'data/dataset/' + MODEL_NAME

In [3]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=IMG_DIMS)

INFO:plaidml:Opening device "metal_amd_radeon_r9_m370x.0"


In [4]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
preds = Dense(2, activation='softmax')(x)

In [ ]:
model = Model(input=base_model.input, outputs=preds)

In [ ]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [ ]:
model.summary()

In [ ]:
def split_train_test_set(directory):
    
    classifiers = next(os.walk(data_set_dir))[1]
    
    print("[INFO] loading images from {}".format(directory))
    # initialize the data and labels
    data = []
    labels = []
    
    # grab the image paths and randomly shuffle them
    image_paths = sorted(list(paths.list_images(directory)))
    random.seed(42)
    random.shuffle(image_paths)
    
    # loop over the input images
    for image_path in image_paths:
        # load the image, pre-process it, and store it in the data list
        image = cv2.imread(image_path)
        image = cv2.resize(image, (IMG_DIMS[1], IMG_DIMS[0]))
        image = img_to_array(image)
        data.append(image)
 
        # extract the class label from the image path and update the
        # labels list        
        label = image_path.split(os.path.sep)[-2]
        l = classifiers.index(label)
        labels.append(l)
        
    # scale the raw pixel intensities to the range [0, 1]
    data = np.array(data, dtype="float") / 255.0
    labels = np.array(labels)
    
    print("[INFO] Splitting the Train and Test sets")
    # partition the data into training and testing splits using 75% of
    # the data for training and the remaining 25% for testing
    train_X, test_X, train_Y, test_Y = train_test_split(data,
                                                        labels,
                                                        test_size=0.2,
                                                        random_state=42)

    # convert the labels from integers to vectors
    train_Y = to_categorical(train_Y, num_classes=2)
    test_Y = to_categorical(test_Y, num_classes=2)

    return train_X, test_X, train_Y, test_Y, classifiers

In [ ]:
def show_model_performance_graph(model, epochs):
    # plot the training loss and accuracy
    plt.style.use("ggplot")
    plt.figure()
    N = epochs
    plt.plot(np.arange(0, N), 
             model.history["loss"], 
             label="train_loss")
    plt.plot(np.arange(0, N), 
             model.history["val_loss"], 
             label="val_loss")
    
    plt.plot(np.arange(0, N), 
             model.history["acc"], 
             label="train_acc")
    plt.plot(np.arange(0, N), 
             model.history["val_acc"], 
             label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.show()

In [ ]:
def train_model(dataset_dir):
    train_X, test_X, train_Y, test_Y, classifiers = split_train_test_set(dataset_dir)
    
    print("[INFO] Augmenting image data...")
    # construct the image generator for data augmentation
    aug = ImageDataGenerator(rotation_range=25, 
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode="nearest")
    
    # initialize the model
    print("[INFO] compiling model...")
    opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
    model.compile(loss="binary_crossentropy",
                  optimizer=opt, 
                  metrics=["accuracy"])
    
    # train the network
    print("[INFO] training network...")
    H = model.fit_generator(
        aug.flow(train_X, train_Y, batch_size=BS),
        validation_data=(test_X, test_Y), 
        steps_per_epoch=len(train_X) // BS,
        epochs=EPOCHS, 
        verbose=1)
    # save the model to disk
    print("[INFO] serializing network...")
    model.save(MODEL_NAME + "_model.h5")
    print("[INFO] serializing label binarizer...")
    
    with open(MODEL_NAME + '_labels.json', 'w') as f:
        for classify in classifiers:
            f.write(classify + '\n')
    
    return H, model, classifiers

In [ ]:
h, model, classifiers = train_model(data_set_dir)

In [ ]:
show_model_performance_graph(h, EPOCHS)

In [ ]:
# load the image
image = cv2.imread('./test2.jpg')
orig = image.copy()

# pre-process the image for classification
image = cv2.resize(image, (IMG_DIMS[0], IMG_DIMS[1]))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

In [ ]:
prediction = model.predict(image)[0]
print(prediction)
idx = np.argmax(prediction)
label = classifiers[idx]
print("{}: {:.2}%".format(label, prediction[idx] * 100))

In [ ]:
for i, label in enumerate(classifiers):
    value = prediction[i]
    print(value)
    print("{}: {:.2}%".format(label, value * 100))

In [ ]:
tfjs.converters.save_keras_model(model, MODEL_NAME + "_tfjs_model")